In [1]:
import numpy as np

In [ ]:
def find_split_point(self, opinion_matrix, feature_index, NUMBER_OF_BIN):
        u_f_opinion = opinion_matrix[:, feature_index]
        arg = np.argsort(u_f_opinion)
        u_f_opinion = u_f_opinion[arg]
        #print(u_f_opinion)
        number_of_unknown = np.sum(u_f_opinion == 10000)
        distance = (len(opinion_matrix)-number_of_unknown)//NUMBER_OF_BIN
        split_point_lists = [u_f_opinion[(i+1)*distance] for i in range(NUMBER_OF_BIN-1)]
        split_point_lists = np.unique(split_point_lists)
        return split_point_lists

In [30]:
def getRatingMatrix(filename):
    # Open the file for reading data
    file = open(filename, "r")

    while 1:
    # Read all the lines from the file and store it in lines
        lines = file.readlines(1000000000)

        # if Lines is empty, simply break
        if not lines:
            break

        # Create a list to hold all the data
        data = []
        data_fo = []

        print("Number of Lines: ", len(lines))

        # For each Data Entry, get the rating and the f-o pairs in their respective list
        for line in lines:
            # print("dealing with new line")
            # Get all the attributes by splitting on ','
            list1 = line.split("\n")[0].split(",")
            list2 = list1.pop()
            # list1 store userid itemid and rating
            # list2 store all f-o pair for each userid
            list2 = list2.split(" ")
            list2 = [int(j) for j in list2]
            list1 = [int(j) for j in list1]
            # Add to the data
            data.append(list1)
            data_fo.append(list2)

        index_f = []
        for i in data_fo:
            index_f.extend(i)

        index_f = np.array(index_f)
        index_f = index_f[np.argmax(index_f)] + 1
        print("Number of feature : ", index_f)
        # convert data into numpy form
        data = np.array(data)

        # Get the indices of the maximum Values in each column
        a = np.argmax(data, axis=0)
        # print(a)
        num_users = data[a[0]][0] + 1
        num_items = data[a[1]][1] + 1

        # print "Max values Indices: ", a
        print("Number of Users: ", num_users)
        print("Number of Items: ", num_items)

        # print(data_fo)
        # print(data)

        ratingMatrix = np.zeros((num_users, num_items), dtype=float)
        opinionMatrix = np.zeros((num_users, index_f), dtype=float)
        opinionMatrix = np.full(opinionMatrix.shape,10000)
        opinionMatrix_I = np.zeros((num_items, index_f), dtype=float)
        opinionMatrix_I = np.full(opinionMatrix_I.shape, 10000)
        # print(len(data))
        # print(len(data_fo))

        for i in range(len(data)):
            list1 = data[i]  # userid itemid rating in line i
            list2 = data_fo[i]  # all f-o pair in line i
            ratingMatrix[list1[0]][list1[1]] = list1[2]
            f_positive = 0
            f_negtive = 0
            for j in range(0, len(list2), 2):
                # list2[j] is feature_id list2[j+1] is value of opinion
                if(list2[j+1]>0):
                    f_positive+=1
                elif(list2[j+1]<0):
                    f_negtive+=1
                opinionMatrix[list1[0]][list2[j]] = f_positive+f_negtive
                opinionMatrix_I[list1[1]][list2[j]] = f_positive-f_negtive
    return opinionMatrix, opinionMatrix_I

In [31]:
op, opi = getRatingMatrix("../data/test_train.txt")

('Number of Lines: ', 1050)
('Number of feature : ', 104)
('Number of Users: ', 1001)
('Number of Items: ', 63)


In [55]:
def getRatingMatrix(filename):
    data = []
    data_fo = []
    feature = []
    with open(filename) as file:
        for line in file:
            d = line[:-1].split(",")
            list1 = [int(x) for x in d[:-1]]
            list2 = [int(x) for x in d[-1].split(" ")]

            data.append(list1)
            data_fo.append(list2)
            for i in list2:
                feature.append(i)
    data = np.array(data)

    num_users = data[:, 0].max() + 1
    num_items = data[:, 1].max() + 1
    num_features = max(feature) + 1
    print num_features
    
    # create rating matrix, and user_opinion, item_opinion matrices
    # user_opinion: user preference for each feature
    # item_opinion: item performance on each feature
    rating_matrix = np.zeros((num_users, num_items), dtype=float)
#     user_opinion = np.zeros((num_users, num_features), dtype=float)
#     item_opinion = np.zeros((num_items, num_features), dtype=float)

    user_opinion = np.full((num_users, num_features), 10000)
    item_opinion = np.full((num_items, num_features), 10000)
    # update the matrices with input data
    # get the accumulated feature opinion scores for users and items.
    for i in range(len(data)):
        user_id, item_id, rating = data[i]
        rating_matrix[user_id][item_id] = rating
        num_pos = 0
        num_neg = 0
        for j in range(0, len(data_fo[i]), 2):
            if data_fo[i][j+1] > 0:
                num_pos += 1
            elif data_fo[i][j+1] < 0:
                num_neg += 1
            user_opinion[user_id][data_fo[i][j]] = num_pos + num_neg
            item_opinion[item_id][data_fo[i][j]] = num_pos - num_neg
            # user_opinion[user_id][data_fo[i][j]] += data_fo[i][j + 1]
            # item_opinion[item_id][data_fo[i][j]] += data_fo[i][j + 1]

    # use the sign function to change the accumulated opinion matrices
    # user_opinion = np.sign(user_opinion)
    # item_opinion = np.sign(item_opinion)

    return rating_matrix, user_opinion, item_opinion

In [56]:
r, op_n, opi_n = getRatingMatrix("../data/test_train.txt")

104


In [57]:
opi_n.shape

(63, 104)

In [60]:
(opi - opi_n).sum()

0

In [ ]:
data = []
data_fo = []
feature = []
with open(filename) as file:
    for line in file:
        d = line[:-1].split(",")
        list1 = [int(x) for x in d[:-1]]
        list2 = [int(x) for x in d[-1].split(" ")]

        data.append(list1)
        data_fo.append(list2)
        for i in list2:
            feature.append(i)
data = np.array(data)

num_users = data[:, 0].max() + 1
num_items = data[:, 1].max() + 1
num_features = max(feature) + 1

# create rating matrix, and user_opinion, item_opinion matrices
# user_opinion: user preference for each feature
# item_opinion: item performance on each feature
rating_matrix = np.zeros((num_users, num_items), dtype=float)
user_opinion = np.zeros((num_users, num_features), dtype=float)
item_opinion = np.zeros((num_items, num_features), dtype=float)
# update the matrices with input data
# get the accumulated feature opinion scores for users and items.
for i in range(len(data)):
    user_id, item_id, rating = data[i]
    rating_matrix[user_id][item_id] = rating
    for j in range(0, len(data_fo[i]), 2):
        user_opinion[user_id][data_fo[i][j]] += data_fo[i][j + 1]
        item_opinion[item_id][data_fo[i][j]] += data_fo[i][j + 1]

# use the sign function to change the accumulated opinion matrices
user_opinion = np.sign(user_opinion)
item_opinion = np.sign(item_opinion)

In [99]:
opinion_matrix = np.random.rand(10, 20)
feature_index = 5
NUM_BIN = 5

In [100]:
u_f_opinion = opinion_matrix[:, feature_index]
arg = np.argsort(u_f_opinion)
u_f_opinion = u_f_opinion[arg]
#print(u_f_opinion)
number_of_unknown = np.sum(u_f_opinion == 10000)
distance = (len(opinion_matrix)-number_of_unknown)//NUM_BIN
split_point_lists = [u_f_opinion[(i+1)*distance] for i in range(NUM_BIN-1)]
split_point_lists = np.unique(split_point_lists)

In [110]:
[u_f_opinion[(i+1)*distance] for i in range(NUM_BIN-1)]

[0.595927038611495, 0.6591785482536219, 0.8545700992797299, 0.945671881407283]

In [113]:
op = opinion_matrix[:, feature_index]
sorted_op = sorted(op)
num_known = np.count_nonzero(~np.isnan(sorted_op))
bin_interval = num_known // NUM_BIN
sp_list = np.unique([sorted_op[(i + 1) * bin_interval] for i in range(NUM_BIN - 1)])

In [114]:
sp_list

array([0.59592704, 0.65917855, 0.8545701 , 0.94567188])

In [112]:
indices_like = np.where((data[:, feature_index] > split_point)&(data[:, feature_index] != 10000))[0]

# Get the indices for the when the opinion value is dislike
indices_dislike = np.where(data[:, feature_index] <= split_point)[0]

# Get the indices for the when the opinion is unknown
indices_unknown = np.where(data[:, feature_index] == 10000)[0]

[0.08713469141553631,
 0.20921604800065396,
 0.595927038611495,
 0.6552661518623754,
 0.6591785482536219,
 0.8470499530863663,
 0.8545700992797299,
 0.9369587645077986,
 0.945671881407283,
 0.953785431554932]

In [116]:
sp_list

array([0.59592704, 0.65917855, 0.8545701 , 0.94567188])

In [126]:
test = np.random.rand(10, 4)

In [134]:
k = np.zeros((10, 4))

In [137]:
k[2][:2] = [1, 2]

In [132]:
test.min()

0.017121629863683263